In [ ]:
#### Import des librairies nécessaires pour le traitement des données, ML, métriques et visualisation

import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    mean_squared_error,
    mean_absolute_error,
    precision_recall_curve
)


In [ ]:
#### Charger le dataset, supprimer les colonnes inutiles et définir les colonnes numériques, catégorielles et ordinales

df = pd.read_csv("churn.csv")  # Adapter le chemin si besoin
df = df.drop(columns=["RowNumber", "CustomerId", "Surname"])

num_features = ["CreditScore", "Age", "Tenure", "Balance", "NumOfProducts", "EstimatedSalary"]
cat_features = ["Geography"]
ordinal_features = ["Gender"]

X = df.drop(columns=["Exited"])
y = df["Exited"]


In [ ]:
#### Définir le pipeline de prétraitement et le modèle de régression logistique
#### Prétraitement : StandardScaler pour numérique, OneHotEncoder pour catégoriel, OrdinalEncoder pour Gender
#### Class_weight = {0:1, 1:3} pour gérer le déséquilibre

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(drop="first"), cat_features),
        ("sex_map", OrdinalEncoder(), ordinal_features),
    ]
)

pipe = Pipeline(
    steps=[
        ("preproc", preprocessor),
        ("clf", LogisticRegression(max_iter=5000, class_weight={0:1,1:3}, random_state=42))
    ]
)


In [ ]:
#### Séparer le dataset en train et test, avec stratification pour maintenir le déséquilibre

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
#### Début du suivi MLflow et entraînement du pipeline
#### Log des paramètres du modèle et de l'expérience

mlflow.set_experiment("Churn_Logistic_Regression")

with mlflow.start_run():
    mlflow.log_param("model", "LogisticRegression")
    mlflow.log_param("max_iter", 5000)
    mlflow.log_param("class_weight", "{0:1,1:3}")
    mlflow.log_param("test_size", 0.2)

    pipe.fit(X_train, y_train)
    y_pred_proba = pipe.predict_proba(X_test)[:,1]


In [ ]:
#### Calcul automatique du seuil optimal pour maximiser le F1-score de la classe minoritaire

precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_proba)
f1_scores = 2 * precisions * recalls / (precisions + recalls + 1e-8)  # éviter division par zéro
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]
y_pred = (y_pred_proba >= best_threshold).astype(int)

mlflow.log_param("best_threshold", best_threshold)


In [ ]:
#### Calcul des métriques principales et log dans MLflow
#### Metrics : Accuracy, Precision, Recall, F1, MSE, MAE

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred_proba)
mae = mean_absolute_error(y_test, y_pred_proba)

mlflow.log_metric("accuracy", acc)
mlflow.log_metric("precision", prec)
mlflow.log_metric("recall", rec)
mlflow.log_metric("f1_score", f1)
mlflow.log_metric("mse", mse)
mlflow.log_metric("mae", mae)


In [ ]:
#### Affichage et sauvegarde de la matrice de confusion
#### Log CSV et PNG pour suivi MLflow

cm = confusion_matrix(y_test, y_pred)
print("=== Matrice de confusion ===")
print(cm)
print("\n=== Classification Report ===")
print(classification_report(y_test, y_pred))

cm_df = pd.DataFrame(cm, index=[f"Actual_{i}" for i in range(cm.shape[0])],
                     columns=[f"Pred_{i}" for i in range(cm.shape[1])])
cm_file = "confusion_matrix.csv"
cm_df.to_csv(cm_file)
mlflow.log_artifact(cm_file)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Prédit")
plt.ylabel("Réel")
plt.title("Matrice de confusion")
plt.tight_layout()
png_file = "confusion_matrix.png"
plt.savefig(png_file)
mlflow.log_artifact(png_file)
plt.close()


In [ ]:
#### Log du modèle entier dans MLflow et affichage du résumé final des métriques

mlflow.sklearn.log_model(pipe, name="logistic_regression_pipeline")

print(f"\nAccuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")
print(f"MSE: {mse:.4f} | MAE: {mae:.4f}")
print(f"Seuil utilisé pour maximiser F1: {best_threshold:.3f}")
